# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [93]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

## 讀取資料

In [94]:
biopsy = dataset("MASS", "biopsy")

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32⍰,Int32,Int32,Int32,Categorical…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [95]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(biopsy)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);
train_dat = biopsy[train_ind, :]
test_dat = biopsy[test_ind, :]

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32⍰,Int32,Int32,Int32,Categorical…
1,640712,1,1,1,1,2,1,2,1,1,benign
2,1297522,2,1,1,1,2,1,1,1,1,benign
3,1105524,4,1,1,1,2,1,1,1,1,benign
4,529329,10,10,10,10,10,10,4,10,10,malignant
5,1186936,2,1,3,2,2,1,2,1,1,benign
6,1257815,5,1,3,1,2,1,2,1,1,benign
7,1198128,10,8,10,10,6,1,3,1,10,malignant
8,1290203,3,1,1,1,2,1,2,1,1,benign
9,1368273,1,1,1,1,2,1,1,1,1,benign


In [96]:
#中位數補值
V1_median = median(skipmissing(train_dat[ : , :V1]))
V2_median = median(skipmissing(train_dat[ : , :V2]))
V3_median = median(skipmissing(train_dat[ : , :V3]))
V4_median = median(skipmissing(train_dat[ : , :V4]))
V5_median = median(skipmissing(train_dat[ : , :V5]))
V6_median = median(skipmissing(train_dat[ : , :V6]))
V7_median = median(skipmissing(train_dat[ : , :V7]))
V8_median = median(skipmissing(train_dat[ : , :V8]))
V9_median = median(skipmissing(train_dat[ : , :V9]))

1.0

In [97]:
train_dat.V1=coalesce.(train_dat[ : ,:V1],V1_median)
train_dat.V2=coalesce.(train_dat[ : ,:V2],V2_median)
train_dat.V3=coalesce.(train_dat[ : ,:V3],V3_median)
train_dat.V4=coalesce.(train_dat[ : ,:V4],V4_median)
train_dat.V5=coalesce.(train_dat[ : ,:V5],V5_median)
train_dat.V6=coalesce.(train_dat[ : ,:V6],V6_median)
train_dat.V7=coalesce.(train_dat[ : ,:V7],V7_median)
train_dat.V8=coalesce.(train_dat[ : ,:V8],V8_median)
train_dat.V9=coalesce.(train_dat[ : ,:V9],V9_median)
test_dat.V1=coalesce.(test_dat[ : ,:V1],V1_median)
test_dat.V2=coalesce.(test_dat[ : ,:V2],V2_median)
test_dat.V3=coalesce.(test_dat[ : ,:V3],V3_median)
test_dat.V4=coalesce.(test_dat[ : ,:V4],V4_median)
test_dat.V5=coalesce.(test_dat[ : ,:V5],V5_median)
test_dat.V6=coalesce.(test_dat[ : ,:V6],V6_median)
test_dat.V7=coalesce.(test_dat[ : ,:V7],V7_median)
test_dat.V8=coalesce.(test_dat[ : ,:V8],V8_median)
test_dat.V9=coalesce.(test_dat[ : ,:V9],V9_median)

140-element Array{Int32,1}:
  1
  1
  1
 10
  1
  1
 10
  1
  1
  3
  1
  1
  1
  ⋮
  1
  2
  1
  1
  1
  1
  1
  1
  1
  1
 10
  1

In [98]:
features = convert(Matrix,train_dat[:,2:10])
labels = string.(train_dat[:,:Class])

559-element Array{String,1}:
 "benign"   
 "benign"   
 "benign"   
 "benign"   
 "benign"   
 "malignant"
 "benign"   
 "malignant"
 "benign"   
 "benign"   
 "benign"   
 "benign"   
 "benign"   
 ⋮          
 "malignant"
 "malignant"
 "malignant"
 "benign"   
 "benign"   
 "benign"   
 "benign"   
 "malignant"
 "benign"   
 "benign"   
 "benign"   
 "benign"   

In [99]:
model = DecisionTree.RandomForestClassifier(n_trees=50, max_depth=2)

RandomForestClassifier
n_trees:             50
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [100]:
DecisionTree.fit!(model, features, labels)

RandomForestClassifier
n_trees:             50
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           2
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             ["benign", "malignant"]
ensemble:            Ensemble of Decision Trees
Trees:      50
Avg Leaves: 3.98
Avg Depth:  2.0

In [101]:
test_features = convert(Matrix,test_dat[:,2:10])
DecisionTree.predict(model, test_features)

140-element Array{String,1}:
 "benign"   
 "benign"   
 "benign"   
 "malignant"
 "benign"   
 "benign"   
 "malignant"
 "benign"   
 "benign"   
 "malignant"
 "malignant"
 "benign"   
 "benign"   
 ⋮          
 "benign"   
 "benign"   
 "malignant"
 "malignant"
 "malignant"
 "malignant"
 "benign"   
 "benign"   
 "benign"   
 "benign"   
 "malignant"
 "malignant"

In [102]:
using ScikitLearn.CrossValidation: cross_val_score
accuracy = cross_val_score(model, features, labels, cv=5)

5-element Array{Float64,1}:
 0.9553571428571429
 0.9821428571428571
 0.9732142857142857
 0.9821428571428571
 0.9459459459459459